In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
train_df = pd.read_csv("train_df.csv", sep=',')
test_df = pd.read_csv("test_df.csv", sep=',')
event_df = pd.read_csv("event_df.csv", sep=',')

In [14]:
print(train_df.head())

   id           date  store_visitors
0   0  2023/1/1 5:00               1
1   1  2023/1/1 6:00               1
2   2  2023/1/1 7:00               0
3   3  2023/1/1 8:00               2
4   4  2023/1/1 9:00              21


In [15]:
print(train_df.tail())

          id             date  store_visitors
13102  13102  2024/6/30 16:00              35
13103  13103  2024/6/30 17:00              20
13104  13104  2024/6/30 18:00               8
13105  13105  2024/6/30 19:00               3
13106  13106  2024/6/30 20:00               0


In [16]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13107 entries, 0 to 13106
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              13107 non-null  int64 
 1   date            13107 non-null  object
 2   store_visitors  13107 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 307.3+ KB
None


In [17]:
print(train_df.describe())

                 id  store_visitors
count  13107.000000    13107.000000
mean    6553.000000       15.286488
std     3783.809324       17.094921
min        0.000000        0.000000
25%     3276.500000        0.000000
50%     6553.000000        7.000000
75%     9829.500000       28.000000
max    13106.000000      119.000000


In [18]:
print("Nombre de lignes :", len(train_df))

Nombre de lignes : 13107


In [19]:
print(train_df.isnull().sum())

id                0
date              0
store_visitors    0
dtype: int64


In [20]:
train_df["date"] = pd.to_datetime(train_df["date"])
print(train_df[train_df['date'].isnull()])
#affiche la ou ça a echoué

Empty DataFrame
Columns: [id, date, store_visitors]
Index: []


In [21]:
train_df = train_df.drop(columns=['id'])

In [22]:
print("Nombre de doublons (True) : ", train_df.duplicated().sum())
print("Nombre de lignes uniques (False) : ", (~train_df.duplicated()).sum())

Nombre de doublons (True) :  0
Nombre de lignes uniques (False) :  13107
